In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import getpass
import pdvega
import plotly.graph_objs as go

from plotly.offline import iplot, init_notebook_mode
import plotly.io as pio
from plotly.graph_objs import *

# for configuring connection 
from configobj import ConfigObj
import os

%matplotlib inline


import os


from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split

#configure the notebook for use in offline mode
init_notebook_mode(connected=True)

In [2]:
df= pd.read_csv("analysis.csv")
df.head()

,Unnamed: 0,hospitalid,sodium,electivesurgery,vent,dialysis,gcs,urine,wbc,temperature,...,m11_True,m12_True,m13_True,m14_True,m15_True,m16_True,m17_True,m18_True,m19_True,m20_True
0,0,59.0,139.0,-1.0,0.0,0.0,15.0,-1.0,14.7,36.1,...,1,0,0,1,1,0,0,0,1,0
1,1,73.0,134.0,-1.0,0.0,0.0,13.0,-1.0,14.1,39.3,...,1,0,0,1,1,0,0,0,1,0
2,2,73.0,-1.0,1.0,1.0,0.0,15.0,-1.0,8.0,34.8,...,0,0,1,0,0,1,0,1,0,0
3,3,63.0,137.0,-1.0,0.0,0.0,15.0,-1.0,10.9,36.6,...,1,0,1,1,1,0,0,1,1,0
4,4,63.0,135.0,-1.0,0.0,0.0,15.0,-1.0,5.9,35.0,...,0,0,1,0,0,0,0,1,0,0


In [4]:
df.shape

(95148, 85)

In [5]:
del df['hospitalid']

df = df.drop(df.columns[[0]], axis=1)
df = df.drop(df.columns[[63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82]], axis=1)

In [6]:
missing_values_count = df.isnull().sum()
#df.replace('-1.0', np.nan)
df = df.replace({-1.0:np.nan, -1.0:np.nan})
df.head()
missing_values_count = df.isnull().sum()
missing_values_count

sodium                        18244
electivesurgery               74997
vent                              0
dialysis                          0
gcs                            1728
urine                         45829
wbc                           22141
temperature                    4139
respiratoryrate                 582
heartrate                       188
meanbp                          263
creatinine                    18332
ph                            73474
hematocrit                    20021
albumin                       58143
pao2                          73474
pco2                          73474
bun                           18774
glucose                       10909
bilirubin                     60797
fio2                          73474
age                            3356
thrombolytics                     0
aids                              0
hepaticfailure                    0
lymphoma                          0
metastaticcancer                  0
leukemia                    

**We moved all the pre-processing including splitting>imputation>Standardization to the CV iterations**

In [8]:
cols_to_norm=['gcs', 'urine', 'wbc', 'sodium',
       'temperature', 'respiratoryrate', 'heartrate', 'meanbp', 'creatinine',
       'ph', 'hematocrit', 'albumin', 'pao2', 'pco2', 'bun', 'glucose',
       'bilirubin', 'fio2', 'age', 'offset']

X=df.drop('destcopy', 1)
y=df['destcopy']
df_cols = list(X)     #fancy impute removes column names.

In [9]:
df.columns

Index(['sodium', 'electivesurgery', 'vent', 'dialysis', 'gcs', 'urine', 'wbc',
       'temperature', 'respiratoryrate', 'heartrate', 'meanbp', 'creatinine',
       'ph', 'hematocrit', 'albumin', 'pao2', 'pco2', 'bun', 'glucose',
       'bilirubin', 'fio2', 'age', 'thrombolytics', 'aids', 'hepaticfailure',
       'lymphoma', 'metastaticcancer', 'leukemia', 'immunosuppression',
       'cirrhosis', 'readmit', 'offset', 'destcopy', 'admitsource_1.0',
       'admitsource_2.0', 'admitsource_3.0', 'admitsource_4.0',
       'admitsource_5.0', 'admitsource_6.0', 'admitsource_7.0',
       'admitsource_8.0', 'diaggroup_ARF', 'diaggroup_Asthma-Emphys',
       'diaggroup_CABG', 'diaggroup_CHF', 'diaggroup_CVA', 'diaggroup_CVOther',
       'diaggroup_CardiacArrest', 'diaggroup_ChestPainUnknown',
       'diaggroup_Coma', 'diaggroup_DKA', 'diaggroup_GIBleed',
       'diaggroup_GIObstruction', 'diaggroup_Neuro', 'diaggroup_Other',
       'diaggroup_Overdose', 'diaggroup_PNA', 'diaggroup_RespMedOther',


**XGB**

In [10]:
from collections import Counter

In [11]:
from sklearn.model_selection import KFold
from sklearn import preprocessing
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import f1_score
from imblearn.metrics import classification_report_imbalanced
from yellowbrick.classifier import ROCAUC
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LogisticRegression
from numpy import loadtxt
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import io 

classes=['Death','Home','Nursing Home','Rehabilitation']



kf = KFold(n_splits=10)


for i in range (1,11):

    for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train = X.iloc[train_index]
        y_train = y.iloc[train_index]  # Based on your code, you might need a ravel call here, but I would look into how you're generating your y
        X_test = X.iloc[test_index]
        y_test = y.iloc[test_index]  # See comment on ravel and  y_train
    
    
    #------------------------------IMPUTE Training Set------------------------------------

        # Use MICE to fill in each row's missing features
        X_train = pd.DataFrame(IterativeImputer(verbose=False, sample_posterior=True).fit_transform(X_train))
        X_train.columns = df_cols

    #------------------------------IMPUTE Testing Set------------------------------------ 

        # Use MICE to fill in each row's missing features
        X_test = pd.DataFrame(IterativeImputer(verbose=False, sample_posterior=True).fit_transform(X_test))
        X_test.columns = df_cols


    #------------------------------Standardize Testing Set------------------------------------

        std_scale = preprocessing.StandardScaler().fit(X_train[cols_to_norm])
        X_train[cols_to_norm] = std_scale.transform(X_train[cols_to_norm])
        X_test[cols_to_norm] = std_scale.transform(X_test[cols_to_norm])
    #------------------------------------------------------------------------------------------

     # Hyperparameters are optimized using hyperopt

        #sm = SMOTE()

        sm = SMOTENC(random_state=50, categorical_features=[1,2,3,22,23,24,25,26,27,28,29,30,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61])        
        X_train_oversampled, y_train_oversampled = sm.fit_sample(X_train, y_train)
        print(sorted(Counter(y_train_oversampled).items()))
        model = XGBClassifier(max_depth=8, gamma=0.063, colsample_bytree=0.71) 
        model.fit(X_train_oversampled, y_train_oversampled)  
        y_pred = model.predict(X_test.values)
        visualizer = ROCAUC(model, classes=classes)
        visualizer.fit(X_train_oversampled, y_train_oversampled)  # Fit the training data to the visualizer
        visualizer.score(X_test.values, y_test)  # Evaluate the model on the test data
        visualizer.poof("XB_SMOTENC_{}_{}.pdf".format(i, fold), clear_figure=True) 
        print(f'For fold {fold}:')
        print(f'Accuracy: {model.score(X_test.values, y_test)}')
        f1=f1_score(y_test, y_pred, average='micro')
        print(f'f-score: {f1}')
        print(classification_report_imbalanced(y_test, y_pred))
        K= classification_report_imbalanced(y_test, y_pred)
        df = pd.read_fwf(io.StringIO(K))
        df.loc["1":"1","pre":"sup"].to_csv("XGB-SMOTENC-D.csv" , sep=',', encoding='utf-8', doublequote=False, index=False, mode="a", header=False)
        df.loc["2":"2","pre":"sup"].to_csv("XGB-SMOTENC-H.csv" , sep=',', encoding='utf-8', doublequote=False, index=False, mode="a", header=False)
        df.loc["3":"3","pre":"sup"].to_csv("XGB-SMOTENC-N.csv" , sep=',', encoding='utf-8', doublequote=False, index=False, mode="a", header=False)
        df.loc["4":"4","pre":"sup"].to_csv("XGB-SMOTENC-R.csv" , sep=',', encoding='utf-8', doublequote=False, index=False, mode="a", header=False)
        df.iloc[6:7,:].to_csv("XGB-SMOTENC-avg.csv" , sep=',', encoding='utf-8', doublequote=False, index=False, mode="a", header=False)

        #


    

C:\Users\llois\Anaconda\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



[(1, 59442), (2, 59442), (3, 59442), (4, 59442)]
For fold 1:
Accuracy: 0.7169732002101944
f-score: 0.7169732002101944
                   pre       rec       spe        f1       geo       iba       sup

          1       0.39      0.48      0.95      0.43      0.68      0.44       642
          2       0.79      0.90      0.41      0.84      0.61      0.39      6776
          3       0.46      0.20      0.95      0.28      0.44      0.18      1716
          4       0.15      0.10      0.98      0.12      0.31      0.09       381

avg / total       0.68      0.72      0.57      0.69      0.57      0.34      9515

[(1, 58698), (2, 58698), (3, 58698), (4, 58698)]
For fold 2:
Accuracy: 0.7043615344193379
f-score: 0.7043615344193379
                   pre       rec       spe        f1       geo       iba       sup

          1       0.47      0.51      0.93      0.49      0.69      0.46       974
          2       0.87      0.79      0.57      0.83      0.67      0.46      7520
          3  

<Figure size 576x396 with 0 Axes>